In [3]:
pip install cdsapi

Keyring is skipped due to an exception: module 'importlib_metadata' has no attribute 'entry_points'
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle as vc 
import cdsapi
from datetime import datetime
#import xlsxwriter 
import csv
import math
import os
import datetime
BASIN = 'EP2018to2020' # set WP, SP, EP, NI, SI
path = os.getcwd() 
path1 = path + '/'+BASIN+'/'

In [8]:
data2 = pd.read_csv(path + '/EPOceanTillLandfall.csv', index_col = 0)
if not os.path.exists(path1):
    os.makedirs(path1)

In [9]:
isoTimeIndex = data2.columns.get_loc('ISO_TIME') 
alldata = []
for val in data2.values:
    isoTime = val[isoTimeIndex]
    date = datetime.datetime.strptime(isoTime, '%Y-%m-%d %H:%M:%S')
    year = date.year
    if year < 2018:
        continue
    alldata.append(val)
dataa = pd.DataFrame(alldata, columns = ['SID', 'ISO_TIME','LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND'])
dataa.to_csv(path1+BASIN+'OceanTillLandfall2018to2020.csv')  

In [10]:
dataa.shape

(409, 7)

In [11]:
import os
import os.path
#Function to check if a number if float or not
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

spread = 4 # Degree spread around center of cyclone

#Function to get u, v, wind and z geopotenital at three pressure levels  for a grid centerd at cyclone location. 
def getPressureData(cdsApi, year, month, date, time, lati, long, path):
  time1 = time.replace(':','_')
  fileName = path+ date +month+year+'_'+time1+'.grib'
  print('fileName  ', fileName)
  if os.path.isfile(fileName):
    print("file already exist")
    return
  cdsApi.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
             'geopotential', 'u_component_of_wind', 'v_component_of_wind',
        ],
        'pressure_level': [
            '225', '500', '700',
        ],
        'year': [
            year,
        ],
        'month': [
            month,
        ],
        'day': [
            date,
        ],
        'time': [
            time,
        ],
        'format': 'grib',
        'area': [
            lati+spread, long-spread, lati-spread,
            long+spread,
        ],
    },
    fileName)
  
#Function to get SST data for grid centerd at cyclone center
def getSSTData(cdsApi, year, month, date, time, lati, long, path):
  time1 = time.replace(':','_')
  fileName = path+ date +month+year+'_'+time1+'.grib'
  print('fileName  ', fileName)
  if os.path.isfile(fileName):
    print("file already exist")
    return
  cdsApi.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': 'sea_surface_temperature',
        'year': year,
        'month': month,
        'day': date,
        'time': time,
        'area': [
            lati+spread, long-spread, lati-spread,
            long+spread,
        ],
    },
    fileName)
  
def createFolder(path):
  if not os.path.exists(path):
    os.makedirs(path)
    
#Rounding lat, long to the multiple of 0.25
def roundF(x, base=.25):
    return base * round(x/base)

In [12]:
data2 = pd.read_csv(path1+BASIN+'OceanTillLandfall2018to2020.csv', index_col=0)
sidIndex = data2.columns.get_loc('SID') 
isoTimeIndex = data2.columns.get_loc('ISO_TIME') 
latIndex = data2.columns.get_loc('LAT') 
longIndex = data2.columns.get_loc('LON')
count = 0
dataValues = data2.values
cdsApi = cdsapi.Client()
for data in dataValues:
  print('count  ', count)
  count = count + 1
  sid = data[sidIndex]
  #Creating folder for pressure data
  folder1 = path1+'data/pressure/'+sid+'/'
  #Creating folder for SST data
  folder2 = path1 +'data/sst/'+sid+'/'
  createFolder(folder1)
  createFolder(folder2)
  isoTime = data[isoTimeIndex]
  lat = data[latIndex]
  long = data[longIndex]
  date = datetime.datetime.strptime(isoTime, '%Y-%m-%d %H:%M:%S')
  year = date.year

  year = str(year)
  month = date.month
  day = date.day
  time = date.hour

  if month < 10:
    month = '0'+str(month)
  else:
    month = str(month)

  if day < 10:
    day = '0'+str(day)
  else:
    day = str(day)

  if time<10:
    time = '0'+str(time)+':00'
  else:
    time = str(time) +':00'

  lat = roundF(lat)
  long = roundF(long)

  #print('day  ', day, ' month ', month, ' year ', year, ' time ', time, ' lat ', lat, ' long ', long)
  getPressureData(cdsApi, year, month, day, time, lat, long, folder1)
  getSSTData(cdsApi, year, month, day, time, lat, long, folder2)

count   0
fileName   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/pressure/2018131N12235/10052018_12_00.grib


2021-02-09 00:57:00,747 INFO Welcome to the CDS
2021-02-09 00:57:00,748 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2021-02-09 00:57:00,963 INFO Request is completed
2021-02-09 00:57:00,964 INFO Downloading https://download-0008.copernicus-climate.eu/cache-compute-0008/cache/data2/adaptor.mars.internal-1612803013.8507423-9717-3-89dfaf85-624b-4265-8d03-f07e9e610870.grib to /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/pressure/2018131N12235/10052018_12_00.grib (21.1K)
2021-02-09 00:57:01,844 INFO Download rate 24K/s    
2021-02-09 00:57:02,193 INFO Welcome to the CDS
2021-02-09 00:57:02,194 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


fileName   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/sst/2018131N12235/10052018_12_00.grib


2021-02-09 00:57:02,414 INFO Downloading https://download-0012.copernicus-climate.eu/cache-compute-0012/cache/data8/adaptor.mars.internal-1612803019.5565164-14849-5-0861a1f9-c571-4da5-9c9e-12ba4cee7800.grib to /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/sst/2018131N12235/10052018_12_00.grib (2.3K)
2021-02-09 00:57:03,129 INFO Download rate 3.3K/s
2021-02-09 00:57:03,503 INFO Welcome to the CDS
2021-02-09 00:57:03,504 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


count   1
fileName   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/pressure/2018157N14255/06062018_00_00.grib


...

2021-02-09 01:12:25,824 INFO Downloading https://download-0002.copernicus-climate.eu/cache-compute-0002/cache/data1/adaptor.mars.internal-1612807870.823226-13813-9-15696726-35b2-4e73-8315-6ca688865e8c.grib to /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/sst/2020307N12250/01112020_18_00.grib (2.3K)
2021-02-09 01:12:26,525 INFO Download rate 3.3K/s
2021-02-09 01:12:26,868 INFO Welcome to the CDS
2021-02-09 01:12:26,870 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


count   408
fileName   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/pressure/2020318N12257/12112020_18_00.grib


2021-02-09 01:12:27,100 INFO Downloading https://download-0003.copernicus-climate.eu/cache-compute-0003/cache/data7/adaptor.mars.internal-1612807873.079617-16060-3-280f9855-7419-43ef-9fc9-62506f275653.grib to /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/pressure/2020318N12257/12112020_18_00.grib (21.1K)
2021-02-09 01:12:28,040 INFO Download rate 22.5K/s  
2021-02-09 01:12:28,378 INFO Welcome to the CDS
2021-02-09 01:12:28,379 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


fileName   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/sst/2020318N12257/12112020_18_00.grib


2021-02-09 01:12:28,594 INFO Downloading https://download-0007.copernicus-climate.eu/cache-compute-0007/cache/data5/adaptor.mars.internal-1612807877.4782798-23247-26-02df71fb-a8c3-4916-9ceb-87fc2c8d1a9d.grib to /Users/sandeepkumar/Desktop/Landfall/ICMLWork/DataAndCode/EP_2018to2020CompleteDataAndCode/EP2018to2020/data/sst/2020318N12257/12112020_18_00.grib (2.3K)
2021-02-09 01:12:28,777 INFO Download rate 12.8K/s


In [22]:
#Function that return the .grib file.
def getFileName(folder, isoTime):
    date = datetime.datetime.strptime(isoTime, '%Y-%m-%d %H:%M:%S')

    year = date.year
    year = str(year)
    month = date.month
    day = date.day
    time = date.hour

    if month < 10:
      month = '0'+str(month)
    else:
      month = str(month)

    if day < 10:
      day = '0'+str(day)
    else:
      day = str(day)

    if time<10:
      time = '0'+str(time)+'_00'
    else:
      time = str(time) +'_00'

    file1 = folder+ day +month+year+'_'+time+'.grib'
    return file1

In [15]:
pip install eccodes

Keyring is skipped due to an exception: module 'importlib_metadata' has no attribute 'entry_points'
Note: you may need to restart the kernel to use updated packages.


In [23]:
#Reading .grib files and extracting u, v, z fileds and SST for 33*33 grid.
#For each filed we have created a column for field values at each cell of grid. So for each fields we have 33*33 = 1089 columns.
import eccodes
from eccodes import *
data2 = pd.read_csv(path1+BASIN+'OceanTillLandfall2018to2020.csv', index_col = 0, header=0)

sidIndex = data2.columns.get_loc('SID') 
isoTimeIndex = data2.columns.get_loc('ISO_TIME') 
latIndex = data2.columns.get_loc('LAT') 
longIndex = data2.columns.get_loc('LON') 
wwindIndex = data2.columns.get_loc('WMO_WIND') 
wpresIndex = data2.columns.get_loc('WMO_PRES') 
d2lIndex = data2.columns.get_loc('DIST2LAND')

sids = pd.unique(data2['SID'])
gridSize = 33
alldata = []

colums = ['SID', 'ISO_TIME','LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND']
check = True
for sid in sids:
  cyclonData = data2[data2['SID'] == sid]
  valuesC = cyclonData.values
  length = len(valuesC)

  folder1 = path1 +'/data/pressure/'+sid+'/'
  folder2 = path1 +'/data/sst/'+sid+'/'
  for i in range(length):
    #print("i   ", i)
    datac = []
    data = valuesC[i]
    isoTime = data[isoTimeIndex]
    lat = data[latIndex]
    long = data[longIndex]
    wwind = data[wwindIndex]
    wpres = data[wpresIndex]
    d2l = data[d2lIndex]

    datac.append(sid)
    datac.append(isoTime)
    datac.append(lat)
    datac.append(long)
    datac.append(wwind)
    datac.append(wpres)
    datac.append(d2l)

    pressureFile = getFileName(folder1, isoTime)
    sstFile = getFileName(folder2, isoTime)
    
    print('pressureFile  ',pressureFile)
    
    id = codes_index_new_from_file(pressureFile, ["shortName",'level'])
    id1 = codes_index_new_from_file(sstFile, ["shortName"])

    names = codes_index_get(id, "shortName")
    levels = codes_index_get(id, "level")

    for name in names:
      #Getting index for each name (u, v, z)
      codes_index_select(id, "shortName", name)
      for level in levels:
        #Getting index for each leval 225, 500, 700
        codes_index_select(id, "level", level)
        gid = codes_new_from_index(id)
        values = codes_get_values(gid)
        length = len(values)
        # Appening values of field for each location of grid. 
        for i in range(length):
            val = np.float32(values[i])
            datac.append(val)
            #Creating dataset columnds name
            if check:
                colums.append(name+str(level)+'_'+str(i))

    #Getting index for SST. 
    codes_index_select(id1, "shortName", 'sst')
    gid1 = codes_new_from_index(id)
    values = codes_get_values(gid)
    length = len(values)
    #Extracting SST for each locatoin of grid. 
    for i in range(length):
        val = np.float32(values[i])
        datac.append(val)
        if check:
            colums.append('sst'+'_'+str(i))
    alldata.append(datac)
    codes_index_release(id)
    codes_index_release(id1)
    check = False
  
dataa = pd.DataFrame(alldata, columns = colums)
dataa.to_csv(path1+BASIN+'pressureSSTData.csv')

pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018131N12235/10052018_12:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018157N14255/06062018_00:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018161N12260/09062018_18:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018161N12260/09062018_21:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018161N12260/10062018_00:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data

pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018218N14255/05082018_12:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018219N15237/06082018_18:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018226N11245/13082018_12:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018227N11145/30082018_03:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018238N13239/25082018_12:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data

pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018268N14253/30092018_15:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018268N14253/30092018_18:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018268N14253/30092018_21:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018268N14253/01102018_00:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018268N14253/01102018_03:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data

pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/04102018_12:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/04102018_15:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/04102018_18:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/04102018_21:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/05102018_00:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to

pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/11102018_06:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/11102018_09:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/11102018_12:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/11102018_15:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018273N12259_0/11102018_18:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to

pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018292N14261/20102018_03:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018292N14261/20102018_06:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018292N14261/20102018_09:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018292N14261/20102018_12:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2018292N14261/20102018_15:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data

pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2019260N12262_19/19092019_15:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2019260N12262_19/19092019_18:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2019260N12262_19/19092019_21:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2019260N12262_19/20092019_00:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2019260N12262_19/20092019_03:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2

pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2020150N09269/29052020_00:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2020150N09269/29052020_03:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2020150N09269/29052020_06:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2020150N09269/29052020_09:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data/pressure/2020150N09269/29052020_12:00.grib
pressureFile   /Users/sandeepkumar/Desktop/Landfall/ICMLWork/CodeForSubmission/EP_2018to2020CompleteDataAndCode/EP2018to2020//data